In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys

sys.path.append('./dashboard')

In [3]:
from typing import List, Union, Literal, TypedDict, Optional
import os

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.exceptions import OutputParserException
from langchain_anthropic import ChatAnthropic
from langchain_community.callbacks import get_openai_callback

In [4]:
from pydantic import (
    BaseModel,
    ValidationError,
    ValidationInfo,
    field_validator,
)
from pydantic.v1 import BaseModel as BaseModelV1
from pydantic.v1 import create_model, validator, ValidationError

In [5]:
from dashboard.model import get_model
from dashboard.plan import get_dashboard_plan

In [6]:
import vizro.plotly.express as px
import vizro.models as vm
from vizro import Vizro
import pandas as pd
from functools import partial
from vizro_ai import VizroAI

In [7]:
from tqdm.notebook import trange, tqdm_notebook

## OpenAI

In [8]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

## Anthropic

In [10]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
    base_url=os.environ.get("ANTHROPIC_API_BASE")
)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model="claude-3-opus-20240229",
)
print(message.content)

[TextBlock(text="Hello! It's nice to meet you. How can I assist you today?", type='text')]


## Choose model

In [10]:
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# model = ChatAnthropic(
#     model='claude-3-opus-20240229',
#     anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"),
#     anthropic_api_url=os.environ.get("ANTHROPIC_BASE_URL")
# )

In [11]:
fig_builder = VizroAI(model=model)

## Starting to construct a page

In [12]:
query = ("I need a page with a bar chart shoing the population per continent "
         "and a scatter chart showing the life expectency per country as a function gdp. "
         "Make a filter on the GDP column and use a dropdown as selector. This filter should only "
         "apply to the bar chart. The bar chart should be a stacked bar chart, while "
         "the scatter chart should be colored by the column 'continent'. I also want "
         "a table that shows the data. The title of the page should be: `This is big time data`. I also want a second page with just "
         "a card on it that links to `https://vizro.readthedocs.io/`. The title of the dashboard should be: `My wonderful "
         "jolly dashboard showing a lot of data`.")

In [13]:
df = px.data.gapminder()

In [14]:
dashboard_plan = get_dashboard_plan(query, model)

In [15]:
dashboard_plan

DashboardPlanner(title='My wonderful jolly dashboard showing a lot of data', pages=[PagePlanner(title='This is big time data', components=Components(components=[Component(component_name='Graph', component_description='A stacked bar chart showing the population per continent.', component_id='population_bar_chart'), Component(component_name='Graph', component_description='A scatter chart showing life expectancy per country as a function of GDP, colored by continent.', component_id='life_expectancy_scatter_chart'), Component(component_name='AgGrid', component_description='A table displaying the data used in the charts.', component_id='data_table')]), controls=Controls(controls=[Control(control_name='Filter', control_description='A dropdown selector to filter data based on GDP values, applicable only to the bar chart.')])), PagePlanner(title='Documentation Link', components=Components(components=[Component(component_name='Card', component_description='A card linking to the Vizro documentat

In [16]:
for i, page in enumerate(dashboard_plan.pages):
    print(f"-----> Page: {page.title}")
    print("----Components----")
    for j in page.components.components:
        print(repr(j))
    # available_components = [comp.component_id for comp in dashboard_plan.pages[0].components.components]
    print("----Controls----")
    for j in page.controls.controls:
        print(repr(j))
    print("============================")


-----> Page: This is big time data
----Components----
Component(component_name='Graph', component_description='A stacked bar chart showing the population per continent.', component_id='population_bar_chart')
Component(component_name='Graph', component_description='A scatter chart showing life expectancy per country as a function of GDP, colored by continent.', component_id='life_expectancy_scatter_chart')
Component(component_name='AgGrid', component_description='A table displaying the data used in the charts.', component_id='data_table')
----Controls----
Control(control_name='Filter', control_description='A dropdown selector to filter data based on GDP values, applicable only to the bar chart.')
-----> Page: Documentation Link
----Components----
Component(component_name='Card', component_description='A card linking to the Vizro documentation.', component_id='vizro_link_card')
----Controls----


In [17]:
# vm.Graph(id = "population_bar_chart",figure=px.bar(df,x = "country"))

In [18]:
class PageBuilder:
    def __init__(self, model, data, page_plan, fig_builder):
        self._model = model
        self._data = data
        self._page_plan = page_plan
        self._fig_builder = fig_builder
        self._components = None
        self._controls = None
        self._page = None

    @property
    def components(self):
        if self._components is None:
            self._components = self._build_components()
        return self._components

    def _build_components(self):
        components = []
        for i in trange(len(self._page_plan.components.components),
                        desc=f"Building components of page: {self._page_plan.title}"):
            components.append(
                self._page_plan.components.components[i].create(fig_builder=self._fig_builder, df=self._data,
                                                                model=model))
        return components

    @property
    def controls(self):
        if self._controls is None:
            self._controls = self._build_controls()
        return self._controls

    @property
    def available_components(self):
        return [comp.id for comp in self.components if comp is not None]

    def _build_controls(self):
        controls = []
        for i in trange(len(self._page_plan.controls.controls),
                        desc=f"Building controls of page: {self._page_plan.title}"):
            controls.append(self._page_plan.controls.controls[i].create(df=self._data, model=self._model,
                                                                        available_components=self.available_components))
        return controls

    @property
    def page(self):
        if self._page is None:
            self._page = vm.Page(
                title=self._page_plan.title,
                components=self.components,
                controls=self.controls
            )
        return self._page



In [19]:
class DashboardBuilder:
    def __init__(self, model, data, dashboard_plan, fig_builder):
        self._model = model
        self._data = data
        self._dashboard_plan = dashboard_plan
        self._fig_builder = fig_builder
        self._pages = None

    @property
    def pages(self):
        if self._pages is None:
            self._pages = self._build_pages()
        return self._pages

    def _build_pages(self):
        pages = []
        for i in trange(len(self._dashboard_plan.pages), desc="Building pages"):
            pages.append(PageBuilder(model=self._model, data=self._data, page_plan=self._dashboard_plan.pages[i],
                                     fig_builder=self._fig_builder).page)
        return pages

    @property
    def dashboard(self):
        return vm.Dashboard(title = dashboard_plan.title, pages=self.pages)

In [20]:
dashboard = DashboardBuilder(model=model, data=df, dashboard_plan=dashboard_plan, fig_builder=fig_builder).dashboard

Building pages:   0%|          | 0/2 [00:00<?, ?it/s]

Building components of page: This is big time data:   0%|          | 0/3 [00:00<?, ?it/s]

Building controls of page: This is big time data:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/Maximilian_Schulz/Library/Application Support/hatch/env/virtual/vizro-ai/fZiNbsWT/vizro-ai/lib/python3.9/site-packages/pydantic/v1/schema.py:631: UserWarning:

CapturedCallable function is excluded from the schema.



Building components of page: Documentation Link:   0%|          | 0/1 [00:00<?, ?it/s]

Building controls of page: Documentation Link: 0it [00:00, ?it/s]

In [21]:
Vizro().build(dashboard).run()